My models consistently reach 1.5k or 2k reward in the first episode, and then fall down to sub 1000 reward.
I followed the basic cnn structure from the tutorials I saw.
Filters gradually increasing, pooling layers between convolutional layers. I didn't use kernal_size of 5 because I figured I was already down_sampling my image.
One major problem was that since I was running this on the colab GPU, I could not run the gym render() function. I had no clue what my model was actually doing in game, and I could only base everything off of reward.

I also couldn't really check if I was feeding data in properly to my neural network, there's a chance I messed up feeding in the reshaped data.

If I had more time, I would mess around with activation of the conv2d layers, and I would try more optimizers.

I would also try changing epsilon value of the model, in retropect, an episolon of 0.1 seems pretty high in a game where one alien bullet will delete your entire run. 

More importantly, I would want to try making more drastic changes to my cnn structure, maybe more conv2d layers before pooling layers, maybe more dense layers. 

Most of the code is taken from the reinforcement learning assignment, I have taken out most of those comments so it will be easier to find my comments.

In [0]:
import gym
import json
import numpy as np
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, MaxPooling2D, MaxPooling1D, Conv2D, Conv1D
from tensorflow.keras.optimizers import Adam, RMSprop, Adagrad

In [0]:
# parameters
epsilon = .1 
num_actions = 6  
max_memory = 500
hidden_size = 100
batch_size = 50
#I don't use the grid_size variable in the end
grid_size = 105*80
print_freq = 10

In [0]:
class ExperienceReplay(object):
    def __init__(self, max_memory=100, discount=.9):
        self.max_memory = max_memory
        self.memory = list()
        self.discount = discount

    def remember(self, states, game_over):
        '''
        Input:
            states: [starting_observation, action_taken, reward_received, new_observation]
            game_over: boolean
        Add the states and game over to the internal memory array. If the array is longer than
        self.max_memory, drop the oldest memory
        '''
        self.memory.append([states, game_over])
        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def get_batch(self, model, batch_size=10):
        '''
        Randomly chooses batch_size memories, possibly repeating.
        For each of these memories, updates the models current best guesses about the value of taking a
            certain action from the starting state, based on the reward received and the model's current
            estimate of how valuable the new state is.
        '''
        len_memory = len(self.memory)
        num_actions = model.output_shape[-1] # the number of possible actions
        # I don't use env_dim in the ned
        # env_dim = self.memory[0][0][0].shape[1] # the number of pixels in the image
        # env_dim = 105*80
        input_size = min(len_memory, batch_size)

        #Instead of inputs = np.zeros((input_size, env_dim)), I did this, I don't know how to use variables in this case
        inputs = np.zeros((input_size, 105, 80))
        targets = np.zeros((input_size, num_actions))
        for i, idx in enumerate(np.random.randint(0, len_memory, size=input_size)):
            starting_observation, action_taken, reward_received, new_observation = self.memory[idx][0]
            game_over = self.memory[idx][1]

            inputs[i] = starting_observation
            
            targets[i] = model.predict(starting_observation.reshape(-1,105,80,1))[0]
            # targets[i] = model.predict(starting_observation)[0]
                               
            if game_over: 
                targets[i, action_taken] = reward_received
            else:
                Q_sa = np.max(model.predict(new_observation.reshape(-1,105,80,1))[0])
                # Q_sa = np.max(model.predict(new_observation)[0])
                targets[i, action_taken] = reward_received + self.discount * Q_sa
        return inputs, targets
        
def build_model():
    '''
     Returns three initialized objects: the model, the environment, and the replay.
    '''
    model = Sequential()

    # This time, I'm making the model use very few filters and very few layers
    model.add(Conv2D(input_shape=(105, 80, 1),
                 filters=8, kernel_size=3, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(filters=16, kernel_size=1, strides=1, padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Flatten())
    model.add(Dense(units=32, activation='relu'))
    model.add(Dense(units=num_actions, activation='relu'))
    model.compile(Adam(), "mse")
    
    # didn't work
    # model.add(Conv2D(input_shape=(105, 80, 1),
    #              filters=8, kernel_size=3, strides=1, padding='same', activation='relu'))
    # model.add(Conv2D(filters=16, kernel_size=1, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.10))
    # model.add(Conv2D(filters=32, kernel_size=1, strides=1, padding='same', activation='relu'))
    # model.add(Conv2D(filters=64, kernel_size=1, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Flatten())
    # model.add(Dense(units=32, activation='relu'))
    # model.add(Dense(units=num_actions, activation='relu'))
    # model.compile(Adam(), "mse")

    # Tried a different optimizer, also tried only using one conv2d layer between pooling layers
    # model.add(Conv2D(input_shape=(105, 80, 1),
    #              filters=16, kernel_size=3, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # # model.add(Dropout(0.10))
    # model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Conv2D(filters=64, kernel_size=1, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Flatten())
    # model.add(Dense(units=64, activation='relu'))
    # model.add(Dense(units=num_actions, activation='relu'))
    # model.compile(RMSprop(), "mse")

    #Tried increasing convolutional layer filters and taking away dense layer units
    # model.add(Conv2D(input_shape=(105, 80, 1),
    #              filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Conv2D(filters=64, kernel_size=1, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Flatten())
    # model.add(Dense(units=32, activation='relu'))
    # model.add(Dense(units=num_actions, activation='relu'))
    # model.compile(Adam(), "mse")

    # Tried making a bunch of pooling layers and adding a ton of filters
    # model.add(Conv2D(input_shape=(105, 80, 1),
    #              filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.20))
    # model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.20))
    # model.add(Conv2D(filters=128, kernel_size=1, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.20))
    # model.add(Conv2D(filters=256, kernel_size=1, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.20))
    # model.add(Flatten())
    # model.add(Dense(units=32, activation='relu'))
    # model.add(Dense(units=num_actions, activation='relu'))
    # model.compile(Adam(), "mse")

    # I tried making the model more simple
    # model.add(Conv2D(input_shape=(105, 80, 1),
    #              filters=16, kernel_size=3, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.20))
    # model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.20))
    # model.add(Flatten())
    # model.add(Dense(units=32, activation='relu'))
    # model.add(Dense(units=num_actions, activation='relu'))
    # model.compile(Adam(), "mse")

    # Tried taking out dropout layers, 
    # model.add(Conv2D(input_shape=(105, 80, 1),
    #              filters=16, kernel_size=3, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))

    # I think kernel_size should be 1 here
    # model.add(Conv2D(filters=32, kernel_size=3, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Flatten())
    # model.add(Dense(units=32, activation='relu'))
    # model.add(Dense(units=num_actions, activation='relu'))
    # model.compile(Adam(), "mse")

    # Tried a different optimizer
    # model.add(Conv2D(input_shape=(105, 80, 1),
    #              filters=16, kernel_size=3, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Conv2D(filters=32, kernel_size=1, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Flatten())
    # model.add(Dense(units=32, activation='relu'))
    # model.add(Dense(units=num_actions, activation='relu'))
    # model.compile(Adagrad(), "mse")
    
    # Define environment/game
    env = gym.make('SpaceInvaders-v0')

    exp_replay = ExperienceReplay(max_memory=max_memory)
    
    return model, env, exp_replay

def take_step(exp_replay, model, starting_observation):
    if np.random.rand() <= epsilon:
        action = np.random.randint(0, num_actions, size=1)
    else:
        q = model.predict(starting_observation.reshape(-1,105,80,1))
        #q = model.predict(starting_observation)
        action = np.argmax(q[0])

    new_observation, reward, game_over, info = env.step(action)
    new_observation = preprocess(new_observation)
    exp_replay.remember([starting_observation, action, reward, new_observation], game_over)

    return new_observation, reward, game_over
    

def train_model(model, env, exp_replay, num_episodes, pretrain_episodes=100):
    '''
    Inputs:
        model, env, and exp_replay objects as returned by build_model
        num_episodes: integer, the number of episodes that should be rolled out for training
    '''
    for episode in range(pretrain_episodes):
        game_over = False
        # get initial input
        starting_observation = env.reset()
        starting_observation = preprocess(starting_observation)
        while not game_over:
            starting_observation, reward, game_over = \
                take_step(exp_replay, model, starting_observation)
    last_total_reward = 0
    total_reward = 0
    for episode in range(1, num_episodes+1):
        loss = 0.
        game_over = False
        # get initial input
        starting_observation = env.reset()
        starting_observation = preprocess(starting_observation)
        while not game_over:
            starting_observation, reward, game_over = \
                take_step(exp_replay, model, starting_observation)
            total_reward += reward
            
        inputs, targets = exp_replay.get_batch(model, batch_size=batch_size)

        loss += model.train_on_batch(inputs.reshape(-1,105,80,1), targets)          
        #loss += model.train_on_batch(inputs, targets)  

        # Print update from this episode
        if episode % print_freq == 0:
            print("Episodes {:04d}-{:04d}/{:04d} | Loss {:.4f} | Total Reward {} | Change in Reward {}".format(
                episode - print_freq + 1, episode, num_episodes, loss, total_reward, total_reward - last_total_reward))
            last_total_reward = total_reward
            total_reward = 0

#I found this code from here https://becominghuman.ai/lets-build-an-atari-ai-part-1-dqn-df57e8ff3b26
def to_grayscale(img):
    return np.mean(img, axis=2).astype(np.uint8)
def downsample(img):
    return img[::2, ::2]
def preprocess(img):
    return to_grayscale(downsample(img))

#Doesn't work with colab
def play_game(model, env):
    last_observation = env.reset()
    last_observation = preprocess(last_observation)
    game_over = False
    total_reward = 0
    while not game_over:
        q = model.predict(last_observation)
        action = np.argmax(q[0])
        last_observation, reward, game_over, info = env.step(action)
        last_observation = preprocess(last_observation)
        total_reward += reward
        env.render()
    print(total_reward)
    env.close()
    


In [5]:
model, env, exp_replay = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 105, 80, 8)        80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 52, 40, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 52, 40, 16)        144       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 26, 20, 16)        0         
_________________________________________________________________
flatten (Flatten)            (None, 8320)              0         
_________________________________________________________________
dense (Dense)                (None, 32)                266272    
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 1

In [6]:
train_model(model, env, exp_replay, num_episodes=500)

Episodes 0001-0010/0500 | Loss 1.4195 | Total Reward 1960.0 | Change in Reward 1960.0
Episodes 0011-0020/0500 | Loss 1.5565 | Total Reward 1615.0 | Change in Reward -345.0
Episodes 0021-0030/0500 | Loss 0.0000 | Total Reward 1130.0 | Change in Reward -485.0
Episodes 0031-0040/0500 | Loss 3.0466 | Total Reward 1335.0 | Change in Reward 205.0
Episodes 0041-0050/0500 | Loss 0.0004 | Total Reward 875.0 | Change in Reward -460.0
Episodes 0051-0060/0500 | Loss 0.0000 | Total Reward 630.0 | Change in Reward -245.0
Episodes 0061-0070/0500 | Loss 0.3333 | Total Reward 740.0 | Change in Reward 110.0
Episodes 0071-0080/0500 | Loss 1.7500 | Total Reward 575.0 | Change in Reward -165.0
Episodes 0081-0090/0500 | Loss 2.0833 | Total Reward 690.0 | Change in Reward 115.0
Episodes 0091-0100/0500 | Loss 0.8181 | Total Reward 1200.0 | Change in Reward 510.0
Episodes 0101-0110/0500 | Loss 1.3333 | Total Reward 450.0 | Change in Reward -750.0
Episodes 0111-0120/0500 | Loss 0.0063 | Total Reward 665.0 | Cha

In [0]:
#Doesn't work because env.render() doesn't work
play_game(model, env)

NameError: ignored

In [0]:
#This is code from above, but it works with dense layers
#The input is't reshaped
#I kept the original comments here

class ExperienceReplay(object):
    def __init__(self, max_memory=100, discount=.9):
        self.max_memory = max_memory
        self.memory = list()
        self.discount = discount

    def remember(self, states, game_over):
        '''
        Input:
            states: [starting_observation, action_taken, reward_received, new_observation]
            game_over: boolean
        Add the states and game over to the internal memory array. If the array is longer than
        self.max_memory, drop the oldest memory
        '''
        self.memory.append([states, game_over])
        if len(self.memory) > self.max_memory:
            del self.memory[0]

    def get_batch(self, model, batch_size=10):
        '''
        Randomly chooses batch_size memories, possibly repeating.
        For each of these memories, updates the models current best guesses about the value of taking a
            certain action from the starting state, based on the reward received and the model's current
            estimate of how valuable the new state is.
        '''
        len_memory = len(self.memory)
        num_actions = model.output_shape[-1] # the number of possible actions
        env_dim = self.memory[0][0][0].shape[1] # the number of pixels in the image
        input_size = min(len_memory, batch_size)
        inputs = np.zeros((input_size, env_dim))
        targets = np.zeros((input_size, num_actions))
        for i, idx in enumerate(np.random.randint(0, len_memory, size=input_size)):
            starting_observation, action_taken, reward_received, new_observation = self.memory[idx][0]
            game_over = self.memory[idx][1]

            # Set the input to the state that was observed in the game before an action was taken
            inputs[i] = starting_observation[0]
            
            # Start with the model's current best guesses about the value of taking each action from this state
            targets[i] = model.predict(starting_observation)[0]
            
            # Now we need to update the value of the action that was taken                      
            if game_over: 
                # if the game is over, give the actual reward received
                targets[i, action_taken] = reward_received
            else:
                # if the game is not over, give the reward received (always zero in this particular game)
                # plus the maximum reward predicted for state we got to by taking this action (with a discount)
                Q_sa = np.max(model.predict(new_observation)[0])
                targets[i, action_taken] = reward_received + self.discount * Q_sa
        return inputs, targets
        
def build_model():
    '''
     Returns three initialized objects: the model, the environment, and the replay.
    '''
    model = Sequential()
    model.add(Dense(hidden_size, input_shape=(80,), activation='relu'))
    model.add(Dense(hidden_size, activation='relu'))
    model.add(Dense(hidden_size, activation='relu'))
    model.add(Dense(num_actions))
    
    # model.add(Conv2D(input_shape=(105, 80, 1),
    #              filters=32, kernel_size=5, strides=1, padding='same', activation='tanh'))
    # model.add(Conv2D(filters=32, kernel_size=5, strides=1, padding='same', activation='relu'))
    # model.add(MaxPooling2D(pool_size=(2,2)))
    # model.add(Dropout(0.15))
    # model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='tanh'))
    # model.add(Conv2D(filters=64, kernel_size=3, strides=1, padding='same', activation='relu'))
    # model.add(Flatten())
    # model.add(Dense(units=32, activation='relu'))
    # model.add(Dense(units=num_actions, activation='relu'))
    # model.compile(RMSprop(), "mse")
    model.compile(Adam(), "mse")

    # Define environment/game
    env = gym.make('SpaceInvaders-v0')

    # Initialize experience replay object
    exp_replay = ExperienceReplay(max_memory=max_memory)
    
    return model, env, exp_replay

def take_step(exp_replay, model, starting_observation):
    # get next action
    if np.random.rand() <= epsilon:
        # epsilon of the time, we just choose randomly
        action = np.random.randint(0, num_actions, size=1)
    else:
        # find which action the model currently thinks is best from this state
        q = model.predict(starting_observation)
        action = np.argmax(q[0])

    # apply action, get rewards and new state
    new_observation, reward, game_over, info = env.step(action)
    new_observation = preprocess(new_observation)
    # store experience
    exp_replay.remember([starting_observation, action, reward, new_observation], game_over)

    return new_observation, reward, game_over
    

def train_model(model, env, exp_replay, num_episodes, pretrain_episodes=10):
    '''
    Inputs:
        model, env, and exp_replay objects as returned by build_model
        num_episodes: integer, the number of episodes that should be rolled out for training
    '''
    for episode in range(pretrain_episodes):
        game_over = False
        # get initial input
        starting_observation = env.reset()
        starting_observation = preprocess(starting_observation)
        while not game_over:
            starting_observation, reward, game_over = \
                take_step(exp_replay, model, starting_observation)
    last_total_reward = 0
    total_reward = 0
    for episode in range(1, num_episodes+1):
        loss = 0.
        game_over = False
        # get initial input
        starting_observation = env.reset()
        starting_observation = preprocess(starting_observation)
        while not game_over:
            starting_observation, reward, game_over = \
                take_step(exp_replay, model, starting_observation)
            total_reward += reward
            
        # get data updated based on the stored experiences
        inputs, targets = exp_replay.get_batch(model, batch_size=batch_size)

        # train model on the updated data        
        loss += model.train_on_batch(inputs, targets)  

        # Print update from this episode
        if episode % print_freq == 0:
            print("Episodes {:04d}-{:04d}/{:04d} | Loss {:.4f} | Total Reward {} | Change in Reward {}".format(
                episode - print_freq + 1, episode, num_episodes, loss, total_reward, total_reward - last_total_reward))
            last_total_reward = total_reward
            total_reward = 0

#I found this code from here https://becominghuman.ai/lets-build-an-atari-ai-part-1-dqn-df57e8ff3b26
def to_grayscale(img):
    return np.mean(img, axis=2).astype(np.uint8)

def downsample(img):
    return img[::2, ::2]

def preprocess(img):
    return to_grayscale(downsample(img))
def play_game(model, env):
    #I'm still preprocessing, which is sketch
    last_observation = env.reset()
    last_observation = preprocess(last_observation)
    game_over = False
    total_reward = 0
    while not game_over:
        q = model.predict(last_observation)
        action = np.argmax(q[0])
        last_observation, reward, game_over, info = env.step(action)
        last_observation = preprocess(last_observation)
        total_reward += reward
        env.render()
    print(total_reward)
    env.close()

In [27]:
model, env, exp_replay = build_model()
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 100)               8100      
_________________________________________________________________
dense_16 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_17 (Dense)             (None, 100)               10100     
_________________________________________________________________
dense_18 (Dense)             (None, 6)                 606       
Total params: 28,906
Trainable params: 28,906
Non-trainable params: 0
_________________________________________________________________


This model somehow does way better than the cnn, although I think it still does pretty badly given that it's basically getting a score of 200 before dying. 

In [28]:
train_model(model, env, exp_replay, num_episodes=100)

Episodes 0001-0010/0100 | Loss 0.0000 | Total Reward 1225.0 | Change in Reward 1225.0
Episodes 0011-0020/0100 | Loss 0.0000 | Total Reward 3000.0 | Change in Reward 1775.0
Episodes 0021-0030/0100 | Loss 2.0829 | Total Reward 2385.0 | Change in Reward -615.0
Episodes 0031-0040/0100 | Loss 0.7498 | Total Reward 3190.0 | Change in Reward 805.0
Episodes 0041-0050/0100 | Loss 0.0000 | Total Reward 2475.0 | Change in Reward -715.0
Episodes 0051-0060/0100 | Loss 1.3329 | Total Reward 3060.0 | Change in Reward 585.0
Episodes 0061-0070/0100 | Loss 0.3331 | Total Reward 2820.0 | Change in Reward -240.0
Episodes 0071-0080/0100 | Loss 0.0832 | Total Reward 2425.0 | Change in Reward -395.0
Episodes 0081-0090/0100 | Loss 0.0000 | Total Reward 2320.0 | Change in Reward -105.0
Episodes 0091-0100/0100 | Loss 0.3330 | Total Reward 2615.0 | Change in Reward 295.0
